In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, Dropout
from keras.optimizers import Adam

# Set random seed for reproducibility
np.random.seed(42)

# Load the MNIST dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale and normalize the input images
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

# Define the generator network
generator = Sequential()
generator.add(Dense(7 * 7 * 128, input_dim=100))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((7, 7, 128)))
generator.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(Conv2DTranspose(1, kernel_size=4, strides=2, padding='same', activation='tanh'))

# Define the discriminator network
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=(28, 28, 1)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Conv2D(128, kernel_size=4, strides=2, padding='same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

# Freeze the discriminator's weights during the generator training
discriminator.trainable = False

# Define the GAN (combining the generator and discriminator)
gan = Sequential()
gan.add(generator)
gan.add(discriminator)

# Compile the GAN
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Training hyperparameters
batch_size = 128
epochs = 10000
sample_interval = 200

# Training loop
for epoch in range(epochs):
    # Select a random batch of real images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_images = X_train[idx]

    # Generate a batch of fake images
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_images = generator.predict(noise)

    # Labels for real and fake images
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator (via the GAN model)
    noise = np.random.normal(0, 1, (batch_size, 100))
    g_loss = gan.train_on_batch(noise, real_labels)

    # Print the progress
    if epoch % sample_interval == 0:
        print(f"Epoch {epoch}/{epochs} - D loss: {d_loss[0]:.4f} - D accuracy: {d_loss[1]:.4f} - G loss: {g_loss:.4f}")

        # Save generated images at regular intervals
        if epoch % sample_interval == 0:
            # Generate 25 random noise samples
            noise = np.random.normal(0, 1, (25, 100))
            generated_images = generator.predict(noise)

            # Rescale images to 0-1 range
            generated_images = 0.5 * generated_images + 0.5

            # Plot the generated images
            fig, axs = plt.subplots(5, 5)
            idx = 0
            for i in range(5):
                for j in range(5):
                    axs[i, j].imshow(generated_images[idx, :, :, 0], cmap='gray')
                    axs[i, j].axis('off')
                    idx += 1
            plt.savefig(f"generated_images_epoch_{epoch}.png")
            plt.close()



4/4 [==============================] - 0s 29ms/step
Epoch 0/10000 - D loss: 0.7090 - D accuracy: 0.2461 - G loss: 0.6921
4/4 [==============================] - 0s 29ms/step
Epoch 200/10000 - D loss: 0.4734 - D accuracy: 0.8008 - G loss: 2.0676
4/4 [==============================] - 0s 30ms/step
Epoch 400/10000 - D loss: 0.6245 - D accuracy: 0.6758 - G loss: 1.3227
4/4 [==============================] - 0s 28ms/step
Epoch 600/10000 - D loss: 0.6505 - D accuracy: 0.6172 - G loss: 1.1738
4/4 [==============================] - 0s 34ms/step
Epoch 800/10000 - D loss: 0.5459 - D accuracy: 0.7695 - G loss: 1.1092
4/4 [==============================] - 0s 30ms/step
